# Рябцев Константин ПМ24-6
## Реализованы дополнительные задания 1-5

In [1962]:
import csv
import pickle
import os
from tabulate import *
import datetime

# Функции

In [1949]:
def load_table(*filenames, fmt=None, detect_types=False):
    
    # Проверка указания файла    
    if not filenames:
        raise ValueError("Не указаны файлы для загрузки")
        
    # Проверка существования файла
    for filename in filenames:
        if not os.path.isfile(filename):
            raise FileNotFoundError("Файла с таким названием не существует")

    # Определение формата по первому файлу, если не задан
    if fmt is None:
        _, ext = os.path.splitext(filenames[0])
        if ext.lower() in ('.csv', '.pkl', '.txt'):
            fmt = ext.lower()
        else:
            raise ValueError("Неизвестный формат файла. Используйте расширения .csv, .pkl или .txt")

    all_data = []
    header = None  # Сохранение заголовока (подразумевается, что заголовок есть в каждой таблице)
    reference_width = None  # Число столбцов в первой таблице

    # Проверка файлов на одинаковость формата
    for filename in filenames:
        _, ext = os.path.splitext(filename)
        if fmt !=  ext.lower():
            raise ValueError("Все файлы должны быть одного формата")

        # Загрузка данных из файлов
        data = []
        if fmt == '.csv':
            with open(filename, 'r', newline='', encoding='utf-8') as f:
                reader = list(csv.reader(f))
                #data = reader (Если строки в Excel таблице не слепляются в одну ячейку с delimiter=',', а с delimiter=';' слепляются, то использовать эту строчку).
                # Закоментировать цикл ниже (отметил строчки знаком #), если с delimiter=';' строки в Excel таблице слепляются в одну ячейку. В функции save_table заменить значение delimiter на ',' (подписал, где это нужно сделать).
                #data.extend(el.split(';') for el in line for line in reader)
                for line in reader:  #
                    for el in line:  #
                        new_el = el.split(';')  #
                        data.append(new_el)  #
        elif fmt == '.pkl':
            with open(filename, 'rb') as f:
                data = pickle.load(f)
        elif fmt == '.txt':
            with open(filename, 'r', encoding='utf-8') as f:
                lines = f.readlines()
            data = [line.strip().split('\t') for line in lines if line.strip()]

        # Проверкамналичия данных в файлах
        if not data:
            raise ValueError(f"Файл {filename} пустой")

        current_header = data[0]  # Заголовок первого файла
        body = data[1:]  # Остальные строки

        # Заполнение списка данными из файлов
        if header is None:
            header = current_header
            reference_width = len(header)
            # Проверка строк в первом файле
            for line in body:
                if len(line) != reference_width:
                    raise ValueError(f"Некорректная структура столбцов в файле {filename}")
            all_data.append(header)  
            all_data.extend(body)
        else:
            # Проверка на совпадение заголовков в оставшихся файлах
            if current_header != header:
                raise ValueError(f"Заголовок в файле {filename} не совпадает с заголовками предыдущих файлов")
            # Проверка строк в оставшихся файлах
            for line in body:
                if len(line) != reference_width:
                    raise ValueError(f"Некорректная структура столбцов в файле {filename}")
            all_data.extend(body)
    
    # Определение типа столбцов по надобности
    if detect_types:
        column_types = detect_column_types(all_data)
        return all_data, column_types

    return all_data

In [1576]:
def save_table(data, filename, fmt=None, max_rows=None):
    
    # Проверка наличия данных
    if not data:
        raise ValueError("Нет данных для сохранения")

    # Проверка указания файла для сохранения
    if not filename or not filename.strip():
        raise ValueError("Не указан файл для сохранения")

    # Проверка формата, если не задан
    if fmt is None:
        _, ext = os.path.splitext(filename)
        if ext.lower() in ('.csv', '.pkl', '.txt'):
            fmt = ext.lower()
        else:
            raise ValueError("Неизвестный формат файла. Используйте расширения .csv, .pkl или .txt")

    # Проверка корректности ввода максимального количества строк в одном файле, если оно задано
    if max_rows is not None and max_rows <= 0:
        raise ValueError(f"Параметр max_rows = {max_rows} может быть только положительным")

    # Запись данных в один файл
    if max_rows is None or max_rows >= len(data):
        if fmt == '.csv':
            with open(filename, 'w', newline='', encoding='utf-8') as f:
                writer = csv.writer(f, delimiter=';')  # Если в Excel таблице строка слепляется в одну ячейку, поменять значение delimiter на ','
                writer.writerows(data)
        elif fmt == '.pkl':
            with open(filename, 'wb') as f:
                pickle.dump(data, f)
        elif fmt == '.txt':
            with open(filename, 'w', encoding='utf-8') as f:
                for line in data:
                    f.write('\t'.join(str(el) for el in line) + '\n')
        return

    # Запись данных в несколько файлов
    base, ext = os.path.splitext(filename)  # Базовое имя файла и его расширение
    header = data[0]  # Заголовоки файлов (подразумевается, что заголовок будет в каждом файле)
    total_rows = (len(data) - 1)  # Количество строк без учета заголовка (подразумевается, что в каждом файле количество строк будет = заголовочная строка + max_rows)
    file_count = (total_rows // max_rows) + (1 if total_rows % max_rows else 0)  # Количество файлов на фаходе

    start = 1
    for i in range(1, file_count + 1):
        end = min(start + max_rows, total_rows)
        splited_data = [header]
        if start != end:
            splited_data.extend(data[start:end])
        else:
            splited_data.append(data[start])
        # Создание имени для каждого файла с помощью добавления индекса к названию
        splited_data_filename = f"{base}_{i}{ext}"

        if fmt == '.csv':
            with open(splited_data_filename, 'w', newline='', encoding='utf-8') as f:
                writer = csv.writer(f, delimiter=';')
                writer.writerows(splited_data)
        elif fmt == '.pkl':
            with open(splited_data_filename, 'wb') as f:
                pickle.dump(splited_data, f)
        elif fmt == '.txt':
            with open(splited_data_filename, 'w', encoding='utf-8') as f:
                for line in splited_data:
                    f.write('\t'.join(str(el) for el in line) + '\n')

        start = end

In [1131]:
def get_rows_by_number(filename, start, stop=None, copy_table=False):

    # Импорт данных из файла
    data = load_table(filename)

    # Проверка данных
    if len(data) == 1:
        raise ValueError("Таблица содержит только заголовок")
        
    header = data[0]  # Заголовок
    new_data = [header]  # Добавление заголовка, он не считается строкой данных
    # Проверка корректности "начального" индекса
    if start > len(data) - 1:  # Выситание заголовка 
        raise IndexError("Номер 'начальной' сохраняемой строки превышает количество строк в файле")
    if start <= 0:
        raise IndexError("Номер 'начальной' сохраняемой строки может быть только положительным")

    # Проверка корректности "конечного" индекса
    if stop is None or start == stop:
        new_data.append(data[start])
    elif stop > len(data) - 1:  # Выситание заголовка 
        raise IndexError("Номер 'последней' сохраняемой строки превышает количество строк в файле")
    elif stop <= 0:
        raise IndexError("Номер 'последней' сохраняемой строки может быть только положительным")
    elif stop < start:
        raise IndexError("Номер 'последней' сохраняемой строки не может превышать номер 'начальной' сохраняемой строки")
    else:
        new_data.extend(data[start:(stop+1)])

    # Если не нужно создавать новый файл
    if not copy_table:
        save_table(new_data, filename)
        return
    # Если нужно создать новый файл с копией данных
    else:
        base, ext = os.path.splitext(filename)
        copied_filename = f"{base}_copied{ext}"
        save_table(new_data, copied_filename)
        return

In [1133]:
def get_rows_by_index(filename, indices, copy_table=False):

    if indices == ():
        raise ValueError("Индексы не введены")

    # Импорт данных из файла
    data = load_table(filename)

    # Проверка данных
    if len(data) == 1:
        raise ValueError("Таблица содержит только заголовок")
        
    header = data[0]  # Заголовок
    header_with_ID = ['ID']
    header_with_ID.extend(header)  # Создание нового заголовка с индексами строк
    new_data = [header_with_ID]  # Добавление заголовка, он не считается строкой данных
    
    # Если введён один индекс
    if isinstance(indices, int):
        idx = int(indices)
        # Проверка корректности индексов
        if idx > len(data) - 1:  # Вычитание заголовка 
            raise IndexError("Номер строки превышает количество строк в файле")
        elif idx <= 0:
            raise IndexError("Номер строки может быть только положительным")
        else:
            new_line = [idx]
            new_line.extend(data[idx])  # Добавление нужных строк с индексом в первом столбце таблицы
            new_data.append(new_line)
    else:
        for idx in set(indices):  
            # Проверка корректности индексов
            if idx > len(data) - 1:  # Выситание заголовка 
                raise IndexError("Номер строки превышает количество строк в файле")
            elif idx <= 0:
                raise IndexError("Номер строки может быть только положительным")
            else:
                new_line = [idx]
                new_line.extend(data[idx])  # Добавление нужных строк с индексом в первом столбце таблицы
                new_data.append(new_line)
            
    # Если не нужно создавать новый файл
    if not copy_table:
        save_table(new_data, filename)
        return
    # Если нужно создать новый файл с копией данных
    else:
        base, ext = os.path.splitext(filename)
        copied_filename = f"{base}_copied{ext}"
        save_table(new_data, copied_filename)
        return

In [1955]:
def get_column_types(filename, by_number=True):
    
    # Импорт данных из файла
    data = load_table(filename)

    # Проверка данных
    if len(data) == 1:
        raise ValueError("Таблица содержит только заголовок")
        
    header = data[0]  # Заголовок
    column_count = len(header)  # Количество столбцов в файле

    # Первая строка после заголовка
    first_line = data[1]

    # Функция для определения типа одного значения
    def detect_type(value):
        if value in ("True", "False"):
            return "bool"
        try:
            int(value)
            return "int"
        except ValueError:
            pass
        try:
            float(value)
            return "float"
        except ValueError:
            pass
        try:
            datetime.datetime.fromisoformat(value)
            return "datetime"
        except ValueError:
            pass
        return "str"

    # Определение типа для каждого столбца по первой строки
    column_types = {}
    if by_number:
        for col_idx in range(column_count):
            column_types[col_idx+1] = detect_type(first_line[col_idx])
    else:
        for col_idx in range(column_count):
            column_types[header[col_idx]] = detect_type(first_line[col_idx])

    # Проверка остальных строк на соответствие типу столбца
    for col_idx in range(column_count):
        for line in data[2:]:
            current_type = detect_type(line[col_idx])
            value_type = list(column_types.values())[col_idx]
            if current_type != value_type:
                raise TypeError(f"Разный тип значений в {col_idx}-м столбце")

    return column_types  

In [1957]:
def set_column_types(filename, types_dict, by_number=True):  # Из задания е очень ясно, что должна делать эта функция, так что реализую её по смыслу программы
    
    '''
        Функция принимает файл и словарь с типами столбцов.
        Функция возвращает данные из файла со значениями в столбцах, приведёнными к нужным типам из словаря types_dict.
        Если в словаре types_dict не задан тип столбца, то функция оставляет тип столбца по умолчанию (str).
        Параметр by_number даёт вункции понять, каким образом определены столбцы в словаре types_dict.
    '''
    
    # Импорт данных из файла
    data = load_table(filename)

    # Проверка данных
    if len(data) == 1:
        raise ValueError("Таблица содержит только заголовок")

    # Проверка наличия переданноый значений типов столбцов
    if not types_dict:
        raise ValueError("Значений типов столбцов не введены")
        
    header = data[0]  # заголовок
    column_count = len(header)  # Количество столбцов в файле

    # Сопоставдение каждого индекса столбца с целевым типом столбца
    col_type_map = {}

    if by_number:
        for col_num, col_type in types_dict.items():
            # Проверка введённого словаря на крректность
            if not isinstance(col_num, int):
                raise ValueError("Словарь введён некорректно")
            real_index = col_num - 1
            # Проверка индексов столбцов на корректность
            if real_index < 0 or real_index >= column_count:
                raise IndexError(f"Столбец с индексом {col_num} некорректен (всего столбцов: {n_cols})")
            col_type_map[real_index] = col_type
    else:
        for col_name, col_type in types_dict.items():
            # Проверка введённого словаря на крректность
            if isinstance(col_name, int):
                raise ValueError("Словарь введён некорректно")
            # Провека значений столбцов на корректность
            if col_name not in header:
                raise ValueError(f"В заголовке нет столбца с названием '{col_name}'")
            real_index = header.index(col_name)
            col_type_map[real_index] = col_type

    # Функция приведения значения к нужному типу
    def cast_value(value, to_type):
        if to_type == 'int':
            return int(value)
        elif to_type == 'float':
            return float(value)
        elif to_type == 'bool':
            return str(value).lower() in ('true', '1', 'yes')
        elif to_type == 'datetime':
            return datetime.datetime.fromisoformat(value)
        else:
            return str(value)

    # Присвоение типов значений столбцов
    for line_idx in range(1, len(data)):
        for col_idx in range(column_count):
            # Присвоение нового типа значениям в столбцах
            current_type = col_type_map.get(col_idx, 'str')
            original_value = data[line_idx][col_idx]
            try:
                data[line_idx][col_idx] = cast_value(original_value, current_type)
            except ValueError:
                raise ValueError(
                    f"Не удалось привести значение '{original_value}' в столбце '{header[col_idx]}' к типу {current_type}"
                )

    return data

In [1139]:
def get_values(data, column=1):

    # Проверка наличия данных
    if not data:
        raise ValueError("Нет данных")

    # Проверка данных
    if len(data) == 1:
        raise ValueError("Таблица содержит только заголовок")
    
    header = data[0]  # Заголовок
    column_count = len(header)  # Количество столбцов в файле

    # Определение индекса столбца
    if isinstance(column, int):
        # Проверка корректности индекса введенного столбца
        if column <= 0 or column > column_count:
            raise IndexError(f"Некорректный номер столбца: {column}. Всего столбцов: {column_count}")
        col_idx = column - 1 
    else:
        # Проверка корректности названия введенного столбца
        if column not in header:
            raise ValueError(f"В заголовке нет столбца с названием '{column}'")
        col_idx = header.index(column)

    # Предполагается, что таблица уже типизирована (если вызывалась set_column_types),
    values = [line[col_idx] for line in data[1:]]

    return values

In [1294]:
def get_value(data, column=1):
    
    # Проверка наличия данных
    if not data:
        raise ValueError("Нет данных")

    # Проверка данных
    if len(data) == 1:
        raise ValueError("Таблица содержит только заголовок")
    if len(data) > 2:
        raise ValueError("В таблице должна быть одна строка без учёта заголовка")

    header = data[0]  # Заголовок
    column_count = len(header)  # Количество столбцов в файле

    # Определение индекса столбца
    if isinstance(column, int):
        # Проверка корректности индекса введенного столбца
        if column <= 0 or column > column_count:
            raise IndexError(f"Некорректный номер столбца: {column}. Всего столбцов: {column_count}")
        col_idx = column - 1 
    else:
        # Проверка корректности названия введенного столбца
        if column not in header:
            raise ValueError(f"В заголовке нет столбца с названием '{column}'")
        col_idx = header.index(column)

    # Предполагается, что таблица уже типизирована (если вызывалась set_column_types),
    value = data[1][col_idx]

    return value

In [1302]:
def set_values(data, values, column=1):

    # Проверка наличия данных
    if not data:
        raise ValueError("Нет данных")
    if not values:
        raise ValueError(f"Нет данных о значениях столбца {column}")

    # Проверка данных
    if len(data) == 1:
        raise ValueError("Таблица содержит только заголовок")

    # Проверка корректрости переданных значений в столбце
    if len(data) - 1 != len(values):
        raise ValueError("Количество значений в переданном столбце не совпадает с количеством значений в столбце таблицы")
    
    header = data[0]  # Заголовок
    column_count = len(header)  # Количество столбцов в файле
    new_data = [header]  # Новый список данных

    # Определение индекса столбца
    if isinstance(column, int):
        # Проверка корректности индекса введенного столбца
        if column <= 0 or column > column_count:
            raise IndexError(f"Некорректный номер столбца: {column}. Всего столбцов: {column_count}")
        
        col_idx = column - 1 
    else:
        # Проверка корректности названия введенного столбца
        if column not in header:
            raise ValueError(f"В заголовке нет столбца с названием '{column}'")
        col_idx = header.index(column)

    # Заполнение нового списка данных
    for line_idx, line in enumerate(data[1:]):
        new_line = []
        for idx, el in enumerate(line):
            if idx == col_idx:
                # Проверка на совпадение типа значения с типом столбца
                if type(el) == type(values[line_idx]):
                    new_line.append(values[line_idx])
                else:
                    raise TypeError(f"Тип значения {values[line_idx]} не совпадает с типом столбца {column}")
            else:
                new_line.append(el)
        new_data.append(new_line)

    return new_data

In [1426]:
def set_value(data, value, column=1):

    # Проверка наличия данных
    if not data:
        raise ValueError("Нет данных")
    if not value:
        raise ValueError(f"Нет данный о значении столбца {column}")

    # Проверка данных
    if len(data) == 1:
        raise ValueError("Таблица содержит только заголовок")
    if len(data) > 2:
        raise ValueError("В таблице должна быть одна строка без учёта заголовка")
    
    header = data[0]  # Заголовок
    column_count = len(header)  # Количество столбцов в файле
    new_data = [header]  # Новый список данных

    # Определение индекса столбца
    if isinstance(column, int):
        # Проверка корректности индекса введенного столбца
        if column <= 0 or column > column_count:
            raise IndexError(f"Некорректный номер столбца: {column}. Всего столбцов: {column_count}")
        
        col_idx = column - 1 
    else:
        # Проверка корректности названия введенного столбца
        if column not in header:
            raise ValueError(f"В заголовке нет столбца с названием '{column}'")
        col_idx = header.index(column)

    # Заполнение нового списка данных
    new_line = []
    for el_idx in range(column_count):
        el = data[1][el_idx]
        if el_idx == col_idx:
            # Проверка на совпадение типа значения с типом столбца
            if type(el) == type(value):
                new_line.append(value)
            else:
                raise TypeError(f"Тип значения {value} не совпадает с типом столбца {column}")
        else:
            new_line.append(el)
    new_data.append(new_line)

    return new_data

In [1580]:
def print_table(data):

    # Проверка наличия данных
    if not data:
        raise ValueError("Нет данных")
        
    # Красивый вывод
    print(tabulate(data[1:], data[0], tablefmt="fancy_grid"))

# Дополнительные Функции

In [1598]:
def concat(data1, data2):
    
    # Проверка наличия данных data1
    if not data1:
        raise ValueError(f"Нет данных в {data1}")

    # Проверка данных data1
    if len(data1) == 1:
        raise ValueError(f"Таблица {data1} содержит только заголовок")

    # Проверка наличия данных data2
    if not data2:
        raise ValueError(f"Нет данных в {data2}")

    # Проверка данных data2
    if len(data2) == 1:
        raise ValueError(f"Таблица {data2} содержит только заголовок")

    header1 = data1[0]  # Заголовок data1
    header2 = data2[0]  # Заголовок data2
    column_count1 = len(header1)  # Количество столбцов в data1
    column_count2 = len(header2)  # Количество столбцов в data2

    # Проверка совпадения форматов data1 и data2
    if header1 != header2 or column_count1 != column_count2:
        raise ValueError("Разные форматы таблиц")

    # Создание новых данных
    new_data = []
    new_data.extend(data1)
    new_data.extend(data2[1:])

    return new_data

In [1638]:
def split(data, line_num):

    # Проверка наличия данных
    if not data:
        raise ValueError("Нет данных")

    # Проверка наличия рвзделительной строки
    if not line_num:
        raise ValueError("Не введено значение разделительной строки")

    # Проверка данных
    if len(data) == 1:
        raise ValueError("Таблица содержит только заголовок")

    # Проверка корректности переданного значения строки
    if line_num >= len(data) - 1 or line_num <= 0:
        raise ValueError("Значение разделительной строки введено некорректно")

    header = data[0]  # Заголовок
    
    # Новые таблицы
    data1 = [header]
    data2 = [header]
    data1.extend(data[1:(line_num+1)])
    data2.extend(data[(line_num+1):])

    return data1, data2

In [1814]:
def detect_column_types(data):

    # Проверка наличия данных
    if not data:
        raise ValueError("Нет данных")
        
    # Проверка данных
    if len(data) == 1:
        raise ValueError("Таблица содержит только заголовок")

    header = data[0]  # Заголовок
    column_count = len(data[0])  # Количество строк
    first_data_line = data[1]  # Первая строка после заголовка

    # Функции для определения типа
    def is_int(value: str) -> bool:
        try:
            int(value)
            return True
        except ValueError:
            return False

    def is_float(value: str) -> bool:
        try:
            float(value)
            return True
        except ValueError:
            return False

    def is_bool(value: str) -> bool:
        lower_val = value.strip().lower()
        return lower_val in ("true", "false", "0", "1", "да", "нет")

    def is_date(value: str, date_formats=None) -> bool:
        import datetime
        if date_formats is None:
            date_formats = ["%Y-%m-%d", "%d.%m.%Y"]
        for fmt in date_formats:
            try:
                datetime.datetime.strptime(value, fmt)
                return True
            except ValueError:
                continue
        return False

    # Определение типа по первой строке после заголовка
    detected_types = []
    for val in first_data_line:
        if is_int(val):
            detected_types.append("int")
        elif is_float(val):
            detected_types.append("float")
        elif is_bool(val):
            detected_types.append("bool")
        elif is_date(val):
            detected_types.append("date")
        else:
            detected_types.append("str")

    return detected_types

# Функции load_table(), save_table() и print_table()

In [1964]:
table = [
        ["Name", "Age", "City"],
        ["Alice", "30", "New York"],
        ["Bob", "25", "London"],
        ["Charlie", "35", "Paris"],
        ["David", "40", "Berlin"],
        ["Eve", "28", "Tokyo"]
    ]
table

[['Name', 'Age', 'City'],
 ['Alice', '30', 'New York'],
 ['Bob', '25', 'London'],
 ['Charlie', '35', 'Paris'],
 ['David', '40', 'Berlin'],
 ['Eve', '28', 'Tokyo']]

In [1966]:
table_1 = [["Name", "Age", "City"]]

In [1968]:
# Сохранение в txt без разбивки
save_table(table, "example.txt")
# Загрузка из одного txt
loaded_txt = load_table("example.txt")
print("Загружено из TXT:")
print_table(loaded_txt)
# Сохранение в txt с разбивкой
save_table(table, "example_split.txt", max_rows=1)
# Получаются файлы example_split_1.txt, example_split_2.txt, example_split_3.txt
# Каждый должен содержать заголовок и по одной строке тела.
loaded_split_txt = load_table("example_split_1.txt", "example_split_2.txt", "example_split_3.txt")
print("Загружено из разбитых TXT:")
print_table(loaded_split_txt)

Загружено из TXT:
╒═════════╤═══════╤══════════╕
│ Name    │   Age │ City     │
╞═════════╪═══════╪══════════╡
│ Alice   │    30 │ New York │
├─────────┼───────┼──────────┤
│ Bob     │    25 │ London   │
├─────────┼───────┼──────────┤
│ Charlie │    35 │ Paris    │
├─────────┼───────┼──────────┤
│ David   │    40 │ Berlin   │
├─────────┼───────┼──────────┤
│ Eve     │    28 │ Tokyo    │
╘═════════╧═══════╧══════════╛
Загружено из разбитых TXT:
╒═════════╤═══════╤══════════╕
│ Name    │   Age │ City     │
╞═════════╪═══════╪══════════╡
│ Alice   │    30 │ New York │
├─────────┼───────┼──────────┤
│ Bob     │    25 │ London   │
├─────────┼───────┼──────────┤
│ Charlie │    35 │ Paris    │
╘═════════╧═══════╧══════════╛


In [1970]:
# Сохраняем в CSV без разбивки
save_table(table, "example.csv")
# Загружаем из одного CSV
loaded_csv = load_table("example.csv")
print("Загружено из CSV:")
print_table(loaded_csv)
# Проверка разбиения для CSV
save_table(table, "example_split.csv", max_rows=2)  # Параметр max_rows не учитывает заголовок, то есть в таблице будет 1 строка заголовка + две строки данных
loaded_split_csv = load_table("example_split_1.csv", "example_split_2.csv", "example_split_3.csv")
print("Загружено из разбитых CSV:")
print_table(loaded_split_csv)

Загружено из CSV:
╒═════════╤═══════╤══════════╕
│ Name    │   Age │ City     │
╞═════════╪═══════╪══════════╡
│ Alice   │    30 │ New York │
├─────────┼───────┼──────────┤
│ Bob     │    25 │ London   │
├─────────┼───────┼──────────┤
│ Charlie │    35 │ Paris    │
├─────────┼───────┼──────────┤
│ David   │    40 │ Berlin   │
├─────────┼───────┼──────────┤
│ Eve     │    28 │ Tokyo    │
╘═════════╧═══════╧══════════╛
Загружено из разбитых CSV:
╒═════════╤═══════╤══════════╕
│ Name    │   Age │ City     │
╞═════════╪═══════╪══════════╡
│ Alice   │    30 │ New York │
├─────────┼───────┼──────────┤
│ Bob     │    25 │ London   │
├─────────┼───────┼──────────┤
│ Charlie │    35 │ Paris    │
├─────────┼───────┼──────────┤
│ David   │    40 │ Berlin   │
├─────────┼───────┼──────────┤
│ Eve     │    28 │ Tokyo    │
╘═════════╧═══════╧══════════╛


In [1972]:
# Сохраняем в pickle без разбивки
save_table(table, "example.pkl")
# Загружаем из одного pickle
loaded_pkl = load_table("example.pkl")
print("Загружено из pickle:")
print_table(loaded_pkl)
# Сохраняем в pickle с разбивкой по 2 строки на файл
save_table(table, "example.pkl", max_rows=2)
# Загружаем из двух pickle-файлов, образовавшихся в результате
loaded_split_pkl = load_table("example_1.pkl", "example_3.pkl")
print("Загружено из разбитых pickle:")
print_table(loaded_split_pkl)

Загружено из pickle:
╒═════════╤═══════╤══════════╕
│ Name    │   Age │ City     │
╞═════════╪═══════╪══════════╡
│ Alice   │    30 │ New York │
├─────────┼───────┼──────────┤
│ Bob     │    25 │ London   │
├─────────┼───────┼──────────┤
│ Charlie │    35 │ Paris    │
├─────────┼───────┼──────────┤
│ David   │    40 │ Berlin   │
├─────────┼───────┼──────────┤
│ Eve     │    28 │ Tokyo    │
╘═════════╧═══════╧══════════╛
Загружено из разбитых pickle:
╒════════╤═══════╤══════════╕
│ Name   │   Age │ City     │
╞════════╪═══════╪══════════╡
│ Alice  │    30 │ New York │
├────────┼───────┼──────────┤
│ Bob    │    25 │ London   │
├────────┼───────┼──────────┤
│ Eve    │    28 │ Tokyo    │
╘════════╧═══════╧══════════╛


In [1974]:
loaded_split = load_table("example_split_1.csv", "example_split_1.txt")  # Пример работы системы ошибок

ValueError: Все файлы должны быть одного формата

In [1976]:
loaded_file = load_table('skfaokf')  # Пример работы системы ошибок

FileNotFoundError: Файла с таким названием не существует

In [1978]:
save_table([], 'err.txt')  # Пример работы системы ошибок

ValueError: Нет данных для сохранения

In [1980]:
save_table(table, '')  # Пример работы системы ошибок

ValueError: Не указан файл для сохранения

In [1982]:
save_table(table, 'err.rx')  # Пример работы системы ошибок

ValueError: Неизвестный формат файла. Используйте расширения .csv, .pkl или .txt

In [1984]:
save_table(table, 'err.txt', max_rows=0)  # Пример работы системы ошибок

ValueError: Параметр max_rows = 0 может быть только положительным

# Функция get_rows_by_number()

In [1986]:
get_rows_by_number("example.csv", 2, 3, True)
loaded_csv = load_table("example.csv")
print("Загружено из CSV:")
print_table(loaded_csv)
loaded_copied_csv = load_table("example_copied.csv")
print("Загружено из копированного CSV:")
print_table(loaded_copied_csv)

Загружено из CSV:
╒═════════╤═══════╤══════════╕
│ Name    │   Age │ City     │
╞═════════╪═══════╪══════════╡
│ Alice   │    30 │ New York │
├─────────┼───────┼──────────┤
│ Bob     │    25 │ London   │
├─────────┼───────┼──────────┤
│ Charlie │    35 │ Paris    │
├─────────┼───────┼──────────┤
│ David   │    40 │ Berlin   │
├─────────┼───────┼──────────┤
│ Eve     │    28 │ Tokyo    │
╘═════════╧═══════╧══════════╛
Загружено из копированного CSV:
╒═════════╤═══════╤════════╕
│ Name    │   Age │ City   │
╞═════════╪═══════╪════════╡
│ Bob     │    25 │ London │
├─────────┼───────┼────────┤
│ Charlie │    35 │ Paris  │
╘═════════╧═══════╧════════╛


In [1988]:
get_rows_by_number("example.csv", 4, 3, True)  # Пример работы системы ошибок

IndexError: Номер 'последней' сохраняемой строки не может превышать номер 'начальной' сохраняемой строки

In [1990]:
get_rows_by_number("example.csv", 2, 10, True)  # Пример работы системы ошибок

IndexError: Номер 'последней' сохраняемой строки превышает количество строк в файле

In [1992]:
get_rows_by_number("example.csv", 0, 3, True)  # Пример работы системы ошибок

IndexError: Номер 'начальной' сохраняемой строки может быть только положительным

In [1994]:
get_rows_by_number("example.csv", 2, -1, True)  # Пример работы системы ошибок

IndexError: Номер 'последней' сохраняемой строки может быть только положительным

# Функция get_rows_by_index()

In [1996]:
get_rows_by_index("example.csv", (3, 5, 3, 2, 1), True)
loaded_csv = load_table("example.csv")
print("Загружено из CSV:")
print_table(loaded_csv)
loaded_copied_csv = load_table("example_copied.csv")
print("Загружено из копированного CSV:")
print_table(loaded_copied_csv)

Загружено из CSV:
╒═════════╤═══════╤══════════╕
│ Name    │   Age │ City     │
╞═════════╪═══════╪══════════╡
│ Alice   │    30 │ New York │
├─────────┼───────┼──────────┤
│ Bob     │    25 │ London   │
├─────────┼───────┼──────────┤
│ Charlie │    35 │ Paris    │
├─────────┼───────┼──────────┤
│ David   │    40 │ Berlin   │
├─────────┼───────┼──────────┤
│ Eve     │    28 │ Tokyo    │
╘═════════╧═══════╧══════════╛
Загружено из копированного CSV:
╒══════╤═════════╤═══════╤══════════╕
│   ID │ Name    │   Age │ City     │
╞══════╪═════════╪═══════╪══════════╡
│    1 │ Alice   │    30 │ New York │
├──────┼─────────┼───────┼──────────┤
│    2 │ Bob     │    25 │ London   │
├──────┼─────────┼───────┼──────────┤
│    3 │ Charlie │    35 │ Paris    │
├──────┼─────────┼───────┼──────────┤
│    5 │ Eve     │    28 │ Tokyo    │
╘══════╧═════════╧═══════╧══════════╛


In [1998]:
get_rows_by_index("example.csv", (3, 5, 3, 2, -1), True)  # Пример работы системы ошибок

IndexError: Номер строки может быть только положительным

In [2000]:
get_rows_by_index("example.csv", (0), True)  # Пример работы системы ошибок

IndexError: Номер строки может быть только положительным

In [2002]:
get_rows_by_index("example.csv", (), True)  # Пример работы системы ошибок

ValueError: Индексы не введены

# Функция get_column_types() и set_column_types()

In [2004]:
print(f"Загружены типы столбцов с их заголовками из файла {"example.csv"}:", get_column_types("example.csv", by_number=False))

Загружены типы столбцов с их заголовками из файла example.csv: {'Name': 'str', 'Age': 'int', 'City': 'str'}


In [2006]:
print(f"Загружены типы столбцов с их номерами из файла {"example.csv"}:", get_column_types("example.csv"))

Загружены типы столбцов с их номерами из файла example.csv: {1: 'str', 2: 'int', 3: 'str'}


In [2008]:
table_data = set_column_types('example.txt', {1: 'str', 2: 'float'}, by_number=True)  # Присвоение нового типа значениям в столбцах 
save_table(table_data, 'file.csv')  # Сохранение данных после присвоения нового типа значениям
print(get_column_types('file.csv', by_number=False))  # Как видно, в файле тип значений сохраняется
new_table = load_table('file.csv')  
save_table(new_table, 'file1.txt')
print(get_column_types('file1.txt', by_number=False)) # После пересохранения данных в новый формат тип данных также сохраняется
new_table

{'Name': 'str', 'Age': 'float', 'City': 'str'}
{'Name': 'str', 'Age': 'float', 'City': 'str'}


[['Name', 'Age', 'City'],
 ['Alice', '30.0', 'New York'],
 ['Bob', '25.0', 'London'],
 ['Charlie', '35.0', 'Paris'],
 ['David', '40.0', 'Berlin'],
 ['Eve', '28.0', 'Tokyo']]

In [2010]:
table_data = set_column_types('example.txt', {'name': 'str', 2: 'float'}, by_number=True)  # Пример работы системы ошибок

ValueError: Словарь введён некорректно

In [2012]:
table_data = set_column_types('example.txt', {1: 'str', 2: 'float'}, by_number=False)  # Пример работы системы ошибок

ValueError: Словарь введён некорректно

In [2014]:
table_data = set_column_types('example.txt', {}, by_number=False)  # Пример работы системы ошибок

ValueError: Значений типов столбцов не введены

In [2026]:
set_column_types('example.txt', {1: 'float', 2: 'float'}, by_number=True)  # Пример работы системы ошибок

ValueError: Не удалось привести значение 'Alice' в столбце 'Name' к типу float

# Функция set_values()

In [2028]:
table_data = set_column_types('example.pkl', {1: 'str', 2: 'int'}, by_number=True)
print_table(table_data)
print(f"Выведен второй стоблец из файла 'example.pkl':", get_values(table_data, 2))

╒═════════╤═══════╤══════════╕
│ Name    │   Age │ City     │
╞═════════╪═══════╪══════════╡
│ Alice   │    30 │ New York │
├─────────┼───────┼──────────┤
│ Bob     │    25 │ London   │
├─────────┼───────┼──────────┤
│ Charlie │    35 │ Paris    │
├─────────┼───────┼──────────┤
│ David   │    40 │ Berlin   │
├─────────┼───────┼──────────┤
│ Eve     │    28 │ Tokyo    │
╘═════════╧═══════╧══════════╛
Выведен второй стоблец из файла 'example.pkl': [30, 25, 35, 40, 28]


In [2030]:
table_data = set_column_types('example.pkl', {1: 'str', 2: 'float'}, by_number=True)
print_table(table_data)
print(f"Выведен второй стоблец из файла 'example.pkl':", get_values(table_data, 'Age'))

╒═════════╤═══════╤══════════╕
│ Name    │   Age │ City     │
╞═════════╪═══════╪══════════╡
│ Alice   │    30 │ New York │
├─────────┼───────┼──────────┤
│ Bob     │    25 │ London   │
├─────────┼───────┼──────────┤
│ Charlie │    35 │ Paris    │
├─────────┼───────┼──────────┤
│ David   │    40 │ Berlin   │
├─────────┼───────┼──────────┤
│ Eve     │    28 │ Tokyo    │
╘═════════╧═══════╧══════════╛
Выведен второй стоблец из файла 'example.pkl': [30.0, 25.0, 35.0, 40.0, 28.0]


In [2032]:
print(f"Выведен второй стоблец из файла 'example.pkl':", get_values(table_1, 'Age'))  # Пример работы системы ошибок

ValueError: Таблица содержит только заголовок

In [2034]:
print(f"Выведен второй стоблец из файла 'example.pkl':", get_values(table_data, 'fssfa'))  # Пример работы системы ошибок

ValueError: В заголовке нет столбца с названием 'fssfa'

In [2036]:
print(f"Выведен второй стоблец из файла 'example.pkl':", get_values(table_data, 4))  # Пример работы системы ошибок

IndexError: Некорректный номер столбца: 4. Всего столбцов: 3

# Функция get_value()

In [2038]:
get_rows_by_index("example.csv", (3), True)
data = load_table("example_copied.csv")
print_table(data)
print(f"Выведен второй стоблец из файла 'example_copied.csv':", get_value(data, 2))

╒══════╤═════════╤═══════╤════════╕
│   ID │ Name    │   Age │ City   │
╞══════╪═════════╪═══════╪════════╡
│    3 │ Charlie │    35 │ Paris  │
╘══════╧═════════╧═══════╧════════╛
Выведен второй стоблец из файла 'example_copied.csv': Charlie


In [2040]:
print(f"Выведен первый стоблец из файла 'example_copied.csv':", get_value(data, 'ID'))

Выведен первый стоблец из файла 'example_copied.csv': 3


In [2042]:
print(f"Выведен второй стоблец из файла 'example_copied.csv':", get_value(data, 5))  # Пример работы системы ошибок

IndexError: Некорректный номер столбца: 5. Всего столбцов: 4

In [2044]:
print(f"Выведен второй стоблец из файла 'example_copied.csv':", get_value([], 2)) # Пример работы системы ошибок

ValueError: Нет данных

In [2046]:
print(f"Выведен второй стоблец из файла 'example_copied.csv':", get_value(data, 'gfj'))  # Пример работы системы ошибок

ValueError: В заголовке нет столбца с названием 'gfj'

In [2048]:
print(f"Выведен второй стоблец из файла 'example_copied.csv':", get_value(table_data, 1))  # Пример работы системы ошибок

ValueError: В таблице должна быть одна строка без учёта заголовка

In [2050]:
save_table(table_1, 'err.txt')  # Пример работы системы ошибок
data_1 = load_table('err.txt')
print_table(data_1)
print(data_1)
print(get_value(data_1, 2))

╒════════╤═══════╤════════╕
│ Name   │ Age   │ City   │
╞════════╪═══════╪════════╡
╘════════╧═══════╧════════╛
[['Name', 'Age', 'City']]


ValueError: Таблица содержит только заголовок

# Функция set_values()

In [2052]:
table_data = set_column_types('example.pkl', {1: 'str', 2: 'int'}, by_number=True)
print_table(table_data)
print("Выведены данные с заменённым вторым столбцом:")
print_table(set_values(table_data, [22, 18, 52, 66, 14], 2))

╒═════════╤═══════╤══════════╕
│ Name    │   Age │ City     │
╞═════════╪═══════╪══════════╡
│ Alice   │    30 │ New York │
├─────────┼───────┼──────────┤
│ Bob     │    25 │ London   │
├─────────┼───────┼──────────┤
│ Charlie │    35 │ Paris    │
├─────────┼───────┼──────────┤
│ David   │    40 │ Berlin   │
├─────────┼───────┼──────────┤
│ Eve     │    28 │ Tokyo    │
╘═════════╧═══════╧══════════╛
Выведены данные с заменённым вторым столбцом:
╒═════════╤═══════╤══════════╕
│ Name    │   Age │ City     │
╞═════════╪═══════╪══════════╡
│ Alice   │    22 │ New York │
├─────────┼───────┼──────────┤
│ Bob     │    18 │ London   │
├─────────┼───────┼──────────┤
│ Charlie │    52 │ Paris    │
├─────────┼───────┼──────────┤
│ David   │    66 │ Berlin   │
├─────────┼───────┼──────────┤
│ Eve     │    14 │ Tokyo    │
╘═════════╧═══════╧══════════╛


In [2054]:
print_table(table_data)
print("Выведены данные с заменённым вторым столбцом:")
print_table(set_values(table_data, [22, 18, 52, 66, 14], "Age"))

╒═════════╤═══════╤══════════╕
│ Name    │   Age │ City     │
╞═════════╪═══════╪══════════╡
│ Alice   │    30 │ New York │
├─────────┼───────┼──────────┤
│ Bob     │    25 │ London   │
├─────────┼───────┼──────────┤
│ Charlie │    35 │ Paris    │
├─────────┼───────┼──────────┤
│ David   │    40 │ Berlin   │
├─────────┼───────┼──────────┤
│ Eve     │    28 │ Tokyo    │
╘═════════╧═══════╧══════════╛
Выведены данные с заменённым вторым столбцом:
╒═════════╤═══════╤══════════╕
│ Name    │   Age │ City     │
╞═════════╪═══════╪══════════╡
│ Alice   │    22 │ New York │
├─────────┼───────┼──────────┤
│ Bob     │    18 │ London   │
├─────────┼───────┼──────────┤
│ Charlie │    52 │ Paris    │
├─────────┼───────┼──────────┤
│ David   │    66 │ Berlin   │
├─────────┼───────┼──────────┤
│ Eve     │    14 │ Tokyo    │
╘═════════╧═══════╧══════════╛


In [2056]:
print_table(set_values(table_data, [22, 18, 52], 2))  # Пример работы системы ошибок

ValueError: Количество значений в переданном столбце не совпадает с количеством значений в столбце таблицы

In [2058]:
print_table(set_values(table_data, [22, 18, 52, 14.5, "пять"], 2))  # Пример работы системы ошибок

TypeError: Тип значения 14.5 не совпадает с типом столбца 2

In [2060]:
print_table(set_values(table_data, [], 2))  # Пример работы системы ошибок

ValueError: Нет данных о значениях столбца 2

In [2062]:
print_table(set_values([], [1, 4, 2, 6, 7], 2))  # Пример работы системы ошибок

ValueError: Нет данных

In [2064]:
print_table(set_values(table_data, [1, 4, 2, 6, 7], -1))  # Пример работы системы ошибок

IndexError: Некорректный номер столбца: -1. Всего столбцов: 3

In [2066]:
print_table(set_values(table_data, [1, 4, 2, 6, 7], 'dngj'))  # Пример работы системы ошибок

ValueError: В заголовке нет столбца с названием 'dngj'

# Функция set_value()

In [2068]:
data = set_column_types('example_copied.csv', {2: 'str', 3: 'int'}, by_number=True)
get_rows_by_index("example.csv", (3), True)
print_table(data)
print("Выведены данные с заменённым вторым столбцом:")
print_table(set_value(data, 52, 3))

╒══════╤═════════╤═══════╤════════╕
│   ID │ Name    │   Age │ City   │
╞══════╪═════════╪═══════╪════════╡
│    3 │ Charlie │    35 │ Paris  │
╘══════╧═════════╧═══════╧════════╛
Выведены данные с заменённым вторым столбцом:
╒══════╤═════════╤═══════╤════════╕
│   ID │ Name    │   Age │ City   │
╞══════╪═════════╪═══════╪════════╡
│    3 │ Charlie │    52 │ Paris  │
╘══════╧═════════╧═══════╧════════╛


In [2070]:
print_table(data)
print("Выведены данные с заменённым вторым столбцом:")
print_table(set_value(data, 'Oleg', 'Name'))

╒══════╤═════════╤═══════╤════════╕
│   ID │ Name    │   Age │ City   │
╞══════╪═════════╪═══════╪════════╡
│    3 │ Charlie │    35 │ Paris  │
╘══════╧═════════╧═══════╧════════╛
Выведены данные с заменённым вторым столбцом:
╒══════╤════════╤═══════╤════════╕
│   ID │ Name   │   Age │ City   │
╞══════╪════════╪═══════╪════════╡
│    3 │ Oleg   │    35 │ Paris  │
╘══════╧════════╧═══════╧════════╛


In [2072]:
print_table(set_value(data, 'Oleg', 'sda'))  # Пример работы системы ошибок

ValueError: В заголовке нет столбца с названием 'sda'

In [2074]:
print_table(set_value(data, '', 2))  # Пример работы системы ошибок

ValueError: Нет данный о значении столбца 2

In [2076]:
print_table(set_value([], 'Oleg', 2))  # Пример работы системы ошибок

ValueError: Нет данных

In [2078]:
print_table(set_value(table, 'Oleg', 2))  # Пример работы системы ошибок

ValueError: В таблице должна быть одна строка без учёта заголовка

In [2080]:
print_table(set_value(data, 'Oleg', 3))  # Пример работы системы ошибок

TypeError: Тип значения Oleg не совпадает с типом столбца 3

# Функция concat() 

In [2082]:
table1 = table
table1

[['Name', 'Age', 'City'],
 ['Alice', '30', 'New York'],
 ['Bob', '25', 'London'],
 ['Charlie', '35', 'Paris'],
 ['David', '40', 'Berlin'],
 ['Eve', '28', 'Tokyo']]

In [2084]:
table2 = [['Name', 'Age', 'City'],
 ['Gosha', '56', 'Moscow'],
 ['Irina', '43', 'Toronto']]
table2

[['Name', 'Age', 'City'],
 ['Gosha', '56', 'Moscow'],
 ['Irina', '43', 'Toronto']]

In [2086]:
print_table(table1)
print_table(table2)

╒═════════╤═══════╤══════════╕
│ Name    │   Age │ City     │
╞═════════╪═══════╪══════════╡
│ Alice   │    30 │ New York │
├─────────┼───────┼──────────┤
│ Bob     │    25 │ London   │
├─────────┼───────┼──────────┤
│ Charlie │    35 │ Paris    │
├─────────┼───────┼──────────┤
│ David   │    40 │ Berlin   │
├─────────┼───────┼──────────┤
│ Eve     │    28 │ Tokyo    │
╘═════════╧═══════╧══════════╛
╒════════╤═══════╤═════════╕
│ Name   │   Age │ City    │
╞════════╪═══════╪═════════╡
│ Gosha  │    56 │ Moscow  │
├────────┼───────┼─────────┤
│ Irina  │    43 │ Toronto │
╘════════╧═══════╧═════════╛


In [2088]:
print_table(concat(table1, table2))

╒═════════╤═══════╤══════════╕
│ Name    │   Age │ City     │
╞═════════╪═══════╪══════════╡
│ Alice   │    30 │ New York │
├─────────┼───────┼──────────┤
│ Bob     │    25 │ London   │
├─────────┼───────┼──────────┤
│ Charlie │    35 │ Paris    │
├─────────┼───────┼──────────┤
│ David   │    40 │ Berlin   │
├─────────┼───────┼──────────┤
│ Eve     │    28 │ Tokyo    │
├─────────┼───────┼──────────┤
│ Gosha   │    56 │ Moscow   │
├─────────┼───────┼──────────┤
│ Irina   │    43 │ Toronto  │
╘═════════╧═══════╧══════════╛


In [2090]:
print_table(concat(table1, []))  # Пример работы системы ошибок

ValueError: Нет данных в []

In [2092]:
table2 = table2[1:]
table2

[['Gosha', '56', 'Moscow'], ['Irina', '43', 'Toronto']]

In [2094]:
print_table(concat(table1, table2))  # Пример работы системы ошибок

ValueError: Разные форматы таблиц

In [2096]:
table2 = [['Name', 'Age'],
 ['Gosha', '56'],
 ['Irina', '43']]

In [2098]:
print_table(concat(table1, table2))  # Пример работы системы ошибок

ValueError: Разные форматы таблиц

# Функция split()

In [2100]:
data1, data2 = split(table, 3)
print_table(table)
print("Первая таблица, полученная с помощью split() по 3-й строке:")
print_table(data1)
print("Вторая таблица, полученная с помощью split() по 3-й строке:")
print_table(data2)

╒═════════╤═══════╤══════════╕
│ Name    │   Age │ City     │
╞═════════╪═══════╪══════════╡
│ Alice   │    30 │ New York │
├─────────┼───────┼──────────┤
│ Bob     │    25 │ London   │
├─────────┼───────┼──────────┤
│ Charlie │    35 │ Paris    │
├─────────┼───────┼──────────┤
│ David   │    40 │ Berlin   │
├─────────┼───────┼──────────┤
│ Eve     │    28 │ Tokyo    │
╘═════════╧═══════╧══════════╛
Первая таблица, полученная с помощью split() по 3-й строке:
╒═════════╤═══════╤══════════╕
│ Name    │   Age │ City     │
╞═════════╪═══════╪══════════╡
│ Alice   │    30 │ New York │
├─────────┼───────┼──────────┤
│ Bob     │    25 │ London   │
├─────────┼───────┼──────────┤
│ Charlie │    35 │ Paris    │
╘═════════╧═══════╧══════════╛
Вторая таблица, полученная с помощью split() по 3-й строке:
╒════════╤═══════╤════════╕
│ Name   │   Age │ City   │
╞════════╪═══════╪════════╡
│ David  │    40 │ Berlin │
├────────┼───────┼────────┤
│ Eve    │    28 │ Tokyo  │
╘════════╧═══════╧════════╛


In [2102]:
split(table, 5)  # Пример работы системы ошибок

ValueError: Значение разделительной строки введено некорректно

In [2104]:
split([], 3)  # Пример работы системы ошибок

ValueError: Нет данных

# Модифицированная функция load_table с определением типа столбцов

In [2106]:
data, types = load_table('example.csv', detect_types=True)
print("Загружено из 'example.csv':")
print_table(data)
print("Типы столбцов:", types)

Загружено из 'example.csv':
╒═════════╤═══════╤══════════╕
│ Name    │   Age │ City     │
╞═════════╪═══════╪══════════╡
│ Alice   │    30 │ New York │
├─────────┼───────┼──────────┤
│ Bob     │    25 │ London   │
├─────────┼───────┼──────────┤
│ Charlie │    35 │ Paris    │
├─────────┼───────┼──────────┤
│ David   │    40 │ Berlin   │
├─────────┼───────┼──────────┤
│ Eve     │    28 │ Tokyo    │
╘═════════╧═══════╧══════════╛
Типы столбцов: ['str', 'int', 'str']
